In [1]:
import random
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, TimeDistributed, Dense


** Sequence Generation **

In [2]:
def get_sequence(n_timesteps):
    # create a sequence of random numbers in [0, 1]
    X = np.array([random.random() for _ in range(n_timesteps)])
    limit = n_timesteps/4.0
    y = np.array([0 if x < limit else 1 for x in np.cumsum(X)])
    return X, y

In [3]:
X, y = get_sequence(10)
print(X)
print(y)

[0.7445431  0.93479191 0.85433796 0.18578936 0.41708048 0.63879489
 0.03860261 0.59829415 0.24722    0.46115006]
[0 0 1 1 1 1 1 1 1 1]


** Generate Multiple Sequences **

In [4]:
def get_sequences(n_sequences, n_timesteps):
    seqX, seqY = [], []
    for i in range(n_sequences):
        X, y = get_sequence(n_timesteps)
        seqX.append(X)
        seqY.append(y)
    return np.array(seqX).reshape(n_sequences, n_timesteps, 1) , np.array(seqY).reshape(n_sequences, n_timesteps, 1)

** Define and Compile Model **

In [5]:
# set parameters
n_timesteps = 10
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
None


** Fit the Model **

In [6]:
X, y = get_sequences(50000, n_timesteps)
model.fit(X, y, epochs = 1, batch_size=10)

Epoch 1/1
50000/50000 [==============================] - 32s 646us/step - loss: 0.0499 - acc: 0.9820


** Evaluate the Model **

In [11]:
# evaluate the model 
X, y = get_sequences(100, n_timesteps)
loss, acc = model.evaluate(X, y, verbose=0)
print('Loss: %f, Accuracy: %f' % (loss, acc*100))

Loss: 0.024885, Accuracy: 99.000002


** Make Predictions with the Model **

In [12]:
# make predictions
for _ in range(10):
    X, y = get_sequences(1, n_timesteps)
    yhat = model.predict_classes(X, verbose=0)
    exp, pred = y.reshape(n_timesteps), yhat.reshape(n_timesteps)
    print('y=%s, yhat=%s, correct=%s' % (exp, pred, np.array_equal(exp, pred)))

y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 0 0 0 0 0 1 1], yhat=[0 0 0 0 0 0 0 0 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 0 0 0 0 1 1 1], yhat=[0 0 0 0 0 0 0 1 1 1], correct=True
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
